# Tracking FII flows

Plots the Foreign Institutional Investment flows in/out of Indian capital markets.
Follows the documentation [here](https://github.com/shyams80/plutons/blob/master/docs-R/InvestmentFlowsIndia.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)

options("scipen"=999)
options(stringsAsFactors = FALSE)
source("config.R")
source("goofy/plot.common.R")

#initialize
iflows <- InvestmentFlowsIndia()

In [ ]:
fiiDebt <- iflows$FiiCashMarket() %>%
    filter(SECURITY_TYPE == 'Debt') %>%
    group_by(TIME_STAMP) %>%
    summarize(NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    arrange(TIME_STAMP) %>%
    select(TIME_STAMP, NET) %>%
    collect() %>%
    as.data.frame()

fiiEquity <- iflows$FiiCashMarket() %>%
    filter(SECURITY_TYPE == 'Equity') %>%
    group_by(TIME_STAMP) %>%
    summarize(NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    arrange(TIME_STAMP) %>%
    select(TIME_STAMP, NET) %>%
    collect() %>%
    as.data.frame()

fiiHybrid <- iflows$FiiCashMarket() %>%
    filter(SECURITY_TYPE == 'Hybrid') %>%
    group_by(TIME_STAMP) %>%
    summarize(NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    arrange(TIME_STAMP) %>%
    select(TIME_STAMP, NET) %>%
    collect() %>%
    as.data.frame()


### Debt flows

In [ ]:
firstDate <- first(fiiDebt$TIME_STAMP)
lastDate <- last(fiiDebt$TIME_STAMP)
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=20)

ggplot(fiiDebt, aes(x=TIME_STAMP, y=NET)) +
    theme_economist() +
    geom_line() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='Rs. Crore', color='', title="FII Debt Flows", 
         subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    annotate("text", x=lastDate, y=min(fiiDebt$NET, na.rm=T), label = "@StockViz", 
             hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)